# Поиск аниме по текстовому запросу

Скрипт предназначен для поиска подходящего аниме по текстовому запросу. В нем загружаются данные аниме-фильмов из CSV-файла, извлекаются текстовые документы и их метки, а затем создается модель векторизации документов и обучения на основе метода опорных векторов (LinearSVC). 

Для обработки текстовых данных в скрипте используется модуль "TfidfVectorizer", который преобразует текст в числовой вектор в соответствии с частотами терминов, а также удаляет стоп-слова (частые слова, не несущие особой смысловой нагрузки). 

В конце скрипта, используя обученную модель, производится классификация новых документов на основе ключевых слов, введенных пользователем, и поиск подходящих аниме-фильмов. Результаты выводятся в форматированном HTML-коде с информацией о названии, описании, жанрах, тегах, соответствующих ключевым словам. Скрипт "find_anime.py" удобен для быстрого поиска и сравнения аниме-фильмов и может быть использован любителями и профессионалами в этой области.

In [44]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd

from IPython.display import display, HTML

In [132]:
# Загружаем набор данных из файла 
df = pd.read_csv('../data/raw/raw_findanime.csv', index_col=0, header=)

In [173]:
# Выводим первые 5 объектов
df.head(5)

,Title,Short title,Href,Translation,Description,Count episodes,Genres,Rating,Rating count,World rating,...,Character designers,Characters,Seiyuu,Translators,Tags,Popularity,Reviews,Quotes,Similar anime,Image src
0,Ходячий замок (Howl's Moving Castle: Howl no U...,Ходячий замок,https://findanime.net/hodiachii_zamok,переведено,"«Ходячий замок» — полнометражное аниме, выпуще...",1.0,"['паропанк', 'фэнтези', 'сказка', 'приключения...",4.9,4.9,4.9,...,['Ямасита Акихико'],"['Софи Хаттер', 'Хаул']","['Байсё Тиэко', 'Кимура Такуя']",['ReaniMedia'],[],3.0,"['\xa0 \xa0 ""Ходячий замок Хаула"", в другой тр...","['Прости меня, я такая капуша: ты ждал меня вс...","['Навсикая из Долины Ветров', 'Унесённые призр...",https://static.findanime.net/uploads/pics/00/1...
1,Унесённые призраками (Spirited Away: Sen to Ch...,Унесённые призраками,https://findanime.net/unesennye_prizrakami,переведено,Вместе с родителями маленькая Тихиро переезжае...,1.0,"['приключения', 'мистика', 'драма', 'семейный'...",4.9,5.0,4.9,...,['Андо Масаси'],"['Тихиро Огино', 'Хаку (Унесённые призраками)']","['Тамай Юми', 'Хиираги Руми', 'Ирино Мию', 'Ка...",[],[],3.0,[' Лет 10-12 назад (уж и не помню точно) зашла...,"['Если забудешь своё имя, то не сможешь вернут...","['Ходячий замок', 'Мой сосед Тоторо', 'Тетрадь...",https://static.findanime.net/uploads/pics/00/1...
2,Стальной Алхимик: Братство [ТВ-2] (FullMetal A...,Стальной Алхимик: Братство [ТВ-2],https://findanime.net/stalnoi_alhimik__bratstv...,переведено,Нарушив главный запрет Алхимии и попытавшись в...,68.0,"['война', 'драма', 'комедия', 'паропанк', 'фэн...",4.9,4.9,4.9,...,['Канно Хироки'],[],"['Фудзивара Кэйдзи', 'Ёсино Хироюки (сэйю)', '...","['Ancord', 'MiraiDuB', 'AlFair Studio', 'AniDu...","['Холодное оружие', 'Империи', 'Дружба', 'Учит...",3.0,['\xa0 \xa0Тема сравнения двух версий сериала ...,['- Мне нравится эта вечная зима. Всё либо чер...,['Стальной Алхимик [ТВ-1]'],https://static.findanime.net/uploads/pics/00/1...
3,Волейбол! 2 (Haikyuu!! Second Season: Haikyu!!...,Волейбол! 2,https://findanime.net/voleibol__2__A5678,переведено,"После участия в межшкольном чемпионате, волейб...",25.0,"['комедия', 'сёнэн', 'драма', 'спорт', 'школа']",4.9,4.9,4.9,...,['Кисида Такахиро'],"['Кэйдзи Акаси', 'Хитока Яти', 'Кэндзи Футакут...","['Хосоя Ёсимаса', 'Морикубо Сётаро', 'Нака Хир...","['AniPlay.TV', 'AniStar', 'AniLibria.TV', 'AOS...","['Несколько ГГ', 'Дружба', 'Волейбол', 'Спорти...",3.0,['Вот и дошли руки написать отзыв о втором сез...,[],"['Баскетбол Куроко [ТВ-1]', 'Синяя тюрьма: Блю...",https://static.findanime.net/uploads/pics/00/4...
4,В лесу мерцания светлячков (Into the Forest of...,В лесу мерцания светлячков,https://findanime.net/into_the_forest_of_firef...,переведено,"Маленькая девочка заблудилась в лесу, но вдруг...",1.0,"['мистика', 'сёдзё', 'романтика', 'трагедия', ...",4.8,4.7,4.8,...,[],[],"['Утияма Коки', 'Сакура Аянэ']","['LE-Production', 'AniWave', 'Akari GROUP', 'A...",[],3.0,"['""И максимальной оценки будет мало...""Могу че...",[],"['Тетрадь дружбы Нацумэ', 'Сад изящных слов', ...",https://static.findanime.net/uploads/pics/00/1...


In [172]:
# Загружаем стоп-слова для русского языка
stop_words = [
    "бы",
    "был",
    "была",
    "были",
    "вам",
    "вас",
    "весь",
    "во",
    "вот",
    "все",
    "всей",
    "вы",
    "да",
    "для",
    "его",
    "ее",
    "ей",
    "ею",
    "если",
    "есть",
    "еще",
    "же",
    "за",
    "или",
    "им",
    "их",
    "как",
    "кем",
    "ко",
    "когда",
    "кто",
    "ли",
    "лучше",
    "меня",
    "мне",
    "много",
    "может",
    "мы",
    "на",
    "надо",
    "нас",
    "не",
    "него",
    "нее",
    "нет",
    "них",
    "но",
    "ну",
    "о",
    "об",
    "однако",
    "он",
    "она",
    "они",
    "оно",
    "от",
    "очень",
    "по",
    "под",
    "при",
    "с",
    "со",
    "так",
    "также",
    "такой",
    "там",
    "те",
    "тебя",
    "тем",
    "теперь",
    "то",
    "тогда",
    "того",
    "том",
    "ты",
    "уже",
    "хотя",
    "чего",
    "чей",
    "чем",
    "что",
    "чтобы",
    "эта",
    "эти",
    "это",
    "я",
]

# Образец набора данных
data = []
for i in range(4000):
    anime = {
        "название": df.loc[i, "Title"],
        "жанр": df.loc[i, "Genres"],
        "теги": df.loc[i, "Tags"],
        "описание": str(df.loc[i, "Description"]),
        "цитаты": df.loc[i, "Quotes"],
    }
    data.append(anime)


In [ ]:
# Разбиваем данные на текстовые документы и их метки
documents = []
labels = []
for anime in data:
    # Удаляем стоп-слова из текста
    text = " ".join(
        [word for word in anime["название"].split() if word.lower()
         not in stop_words]
    )
    text += " ".join(
        [word for word in anime["жанр"].split() if word.lower() not in stop_words]
    )
    text += " ".join(
        [word for word in anime["теги"].split() if word.lower() not in stop_words]
    )
    text += " ".join(
        [word for word in anime["описание"].split() if word.lower()
         not in stop_words]
    )
    text += " ".join(
        [word for word in anime["цитаты"].split() if word.lower()
         not in stop_words]
    )
    documents.append(text)
    labels.append(anime["название"])

In [125]:
# Создаём модель для векторизации документов и обучения
vectorizer = TfidfVectorizer()
classifier = LinearSVC()

# Обучаем модель на обучающих данных
train_vectors = vectorizer.fit_transform(documents)
classifier.fit(train_vectors, labels)

LinearSVC()

In [171]:
# Несколько примеров текстовых запросов, по которым нужно найти аниме
keywords_str_list = [
    "историческое франция",
    "ниндзя, кровь",
    "драма экшн интрига америка",
    "школа любовь треугольник",
]

# Предсказание и вывод результата
for i, keywords_str in enumerate(keywords_str_list):
    keywords = keywords_str.split()
    new_document = " ".join(keywords)

    new_vector = vectorizer.transform([new_document])
    predicted_names = classifier.predict(new_vector)
    title = predicted_names[0]
    row = df.loc[df["Title"] == title]
    description = row.iloc[0]["Description"]
    genres = row.iloc[0]["Genres"]
    tags = row.iloc[0]["Tags"]

    # Отображаем описание, обернутое в HTML-теги, для форматирования:
    html_description = '{}. Ключевые слова: {}<p>Название: {}<p>Жанры: {}<p>Теги: {}<p><div style="white-space: pre-line">{}</div>'.format(
        i + 1, keywords, title, genres, tags, description
    )
    display(HTML(html_description))